In [2]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [4]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[
                                html.H1(
                                    'SpaceX Launch Records Dashboard'
                                    ,style={'textAlign': 'center', 'color': '#503D36','font-size': 40}
                                ),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(
                                    id='site-dropdown'
                                    ,options=[
                                        {'label': 'All Sites', 'value': 'ALL'}
                                        ,{'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}
                                        ,{'label': 'KSC LC 39A', 'value': 'KSC LC-39A'}
                                        ,{'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                        ,{'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}
                                    ]
                                    ,value='ALL'
                                    ,placeholder="place holder here"
                                    ,searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(
                                    id='payload_slider'
                                    ,min=0
                                    ,max=10000
                                    ,step=1000
                                    ,value=[min_payload, max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),

                                

                                
])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(
            filtered_df
            ,values='class'
            ,names='Launch Site' 
            ,title='title'
        )
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = filtered_df.groupby('class').count().reset_index()        
        fig = px.pie(
            filtered_df, 
            values='Unnamed: 0', 
            names='class', 
            title='title'
        )
    # return the outcomes piechart for a selected site
    return fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")]
)
def update_payload(site_dropdown, payload_slider):
    if site_dropdown == 'All':
        low, high = payload_slider
        df  = spacex_df
        payload = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[payload]
            ,x="Payload Mass (kg)"
            ,y="class"
            ,color="Booster Version"
            ,size='Payload Mass (kg)'
            ,hover_data=['Payload Mass (kg)']
        )
    else:
        low, high = payload_slider
        df  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        payload = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[payload]
            ,x="Payload Mass (kg)"
            ,y="class"
            ,color="Booster Version"
            ,size='Payload Mass (kg)'
            ,hover_data=['Payload Mass (kg)']
        )
    return fig

Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


SystemExit: 1

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [ ]:
# Run the app
if __name__ == '__main__':
    app.run()